# Edex __: Estimate Melting Points of Inorganic Materials

## Problem Statement and Motivations
In this edex, we'll be testing supervised and unsupervised learning models to link an inorganic material to its property.

#### Goal: 
Estimate the melting temperature using chemical formula of inorganic compound. A database of empirical data from 2 physical chemistry handbooks is provided in accompanying "Melting_Data.csv" file.

#### Motivation: 
Similarly, we'll be using machine learning to establish a quantitative structure-activity relationship in a context where laboratory experiments can take too much time to do brute force testing. However, this time we'll attempt to use unsupervised methods to further inform our regression models and improve their predictive power and fidelity. Approaching problems from this point of view can suggest insight into the physics of the problem, like in cases where one subsection of data follow a vastly different model compared to the rest. 

#### Relevant Materials Science Background:
- Mitigating climate change requires, urgently, the transition away from fossil fuels to renewable carbon-neutral energy sources such as wind and solar.
- Melting temperature is a crucial thermodynamic property needed to understand the underlying processes of metal extraction and to design new materials and industrial processes for less carbon and energy-intensive recycling processes.

For more information, see the following:


#### Papers:

[1]	Seko, Atsuto, et al. "Machine learning with systematic density-functional theory calculations: Application to melting temperatures of single-and binary-component solids." Physical Review B 89.5 (2014): 054303.

[2]	Hong, Qi-Jun, et al. "Melting temperature prediction using a graph neural network model: From ancient minerals to new materials." Proceedings of the National Academy of Sciences 119.36 (2022): e2209630119.

[3]	Guan, Pin-Wen, and Venkatasubramanian Viswanathan. "MeltNet: Predicting alloy melting temperature by machine learning." arXiv preprint arXiv:2010.14048 (2020).


## Framing the Problem in ML

This problem explores combined unsupervised and supervised learning. As in the previous edex, we'll be using regression models for the final prediction since we need the output to be a continuous value. Clustering is used to identify groups of similar objects in datasets. This can be as simple as K-means clustering or as unpredictable as t-SNE, a stochastic embedding routine famous for some fun visuals. In order to use regression or clustering for our purposes here, we will need to vectorize an inorganic compound/crystal to construct a numerical input for our models. 

In this example, we already provided some part of materials fingerprint to encode chemical compositions. We will work through the rest of this notebook to construct the rest of the fingerprint. The result will be a vector of a prescribed length with continuous values.

### Import libraries

Begin by importing the necessary python packages. If any are not installed, check their documentation for installation guides using either pip or conda. 

For additional models, you will need to import additional libraries of your own choosing. 

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error

from matminer.featurizers.composition import ElementProperty
from matminer.utils.data import PymatgenData
from pymatgen.core import Composition

## Data Preprocessing

Import the data from "Melting_Data.csv" file. This file contains melting temperatures for a few hundred binary inorganic materials (more on this below). 

Make sure your csv file is in the same folder as this notebook file so you can directly import it. 

In [ ]:
data = pd.read_csv('Melting_Data.csv')
data.head(5)

Column 0: Compound formula\
Column 1: Materials Project ID of the compound/crystal (more below)\
<---- Below are different options for your output. You may want to predict melting temperature either in Kelvin or Celsius. You can also try to predict the log of melting temperatures but please be extra careful when doing the error analysis and model evaluation there as additional step(s) will be required. ---->\
Column 2: Melting temperature in Celsius\
Column 3: Melting temperature in Kelvin\
Column 4: Log10 of melting temperature in Kelvin\
<---- Below are features that we find important for melting point prediction. Some of those are directly extracted from the Materials Project and others are manually computed through a quantum mechanical method called [Density Functional Theory](https://en.wikipedia.org/wiki/Density_functional_theory). ---->\
Columns 5-9: Properties of the crystal (cohesive energy, bulk and shear moduli, formation energy, density)\
Column 10: Ionic character of a bond in the crystal according to Pauling's formula ($\%\ ionic\ character = (1 - e^{-\Delta EN^2 / 4}) \times 100\%$, where $\Delta EN$ is the electronegativity difference between constituting atoms)

The [Materials Project](https://materialsproject.org/) is an open-access database offering material properties. From its documentation: "Every database needs a unique key which can be used to distinguish one entry from another. In the Materials Project, each unique material is given a material_id (also referred to in various places as mp-id, mpid, MPID). This allows a specific polymorph of a given material to be referenced." For instance, [here](https://materialsproject.org/materials/mp-353) you can see the mp-353 entry that corresponds to the most stable Ag2O crystal structure.

# Construct fingerprints for each material

Now we want to make a fixed-length vector format for each crystal for regression/clustering operations to go smoothly. There are many ways to get a fingerprint for a material and here we will show an example of what the material fingerprint can consist of. You are free to add, delete or modify any fingerprint entry (and this is where the beauty of feature engineering is!). 

We will make use of matminer. [Matminer](https://matminer.readthedocs.io/en/latest/) is an open-source Python library for performing data mining and analysis in the field of materials science.

We aim to add the following atomic descriptors to the material fingerprint: periodic row and group numbers, atomic mass, atomic radius, boiling and melting points, and electronegativities. Because our database consists of binary materials, we will just include the (composition-agnostic) property statistics for separate atoms. In the end, for each descriptor we will have 2 added entries into the fingerprint: mean and standard deviation of the descriptor. Here we make use of Pymatgen to get the required data, more on this and other fingerprinting methods can be found in matminer's documentation.

In [ ]:
data["composition"] = data['formula'].map(lambda x: Composition(x))

descriptors = ['row', 'group', 'atomic_mass', 'atomic_radius', 
               'boiling_point', 'melting_point', 'X']
stats = ["mean", "std_dev"]

ep = ElementProperty(data_source=PymatgenData(), features=descriptors, stats=stats)
data = ep.featurize_dataframe(data, "composition")

# deleting an unnecessary column
data = data.drop(columns=['composition'])

In [ ]:
data.head(5)

Now we will use columns starting at `coh_en` and to the right as materials fingerprint and will use the `melt_temp_K` column as an output.

Create a vector of material fingerprints and a vector of outputs and check whether their dimensions match.

Show the dimension of the fingerprint and output the whole fingerprint corresponding to Ag2O.

#### <font color='RED'>YOUR SOLUTION:</font> 

Now we'll split the dataset into a training set and a testing set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

## Example: Linear Regression
As in the regression EdEx, let's build a simple linear regression model to see how the simplest case performs. Remember to heck both training and testing scores. 

#### <font color='RED'>YOUR SOLUTION:</font> 

#### <font color='RED'>YOUR SOLUTION:</font> 

To see how the model's performance looks on a more detailed level, we can try parity plots and/or getting a histogram of deviations between model prediction and actual values.

In [ ]:
y_pred_train = linreg.predict(X_train)
y_pred_test = linreg.predict(X_test)

Find mean absolute error (MAE) and root mean squared error (RMSE) for both training and test sets.

#### <font color='RED'>YOUR SOLUTION:</font> 

A parity plot is just a scatter of $y_{pred}$ versus $y_{true}$. These should ideally form a line along the y=x diagonal, meaning that every prediction is more or less spot-on with the actual value, but you will have some fluctuations. 

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot([-500,4500], [-500,4500], color='black')
plt.scatter(y_train, y_pred_train, color='blue', alpha =0.5, label='train')
plt.scatter(y_test, y_pred_test, color='red', alpha =0.5, label='test')
plt.xlim([-500,4500])
plt.ylim([-500,4500])
plt.xlabel('$y_{true}$', size=16)
plt.ylabel('$y_{pred}$', size=16)
plt.legend(fontsize=16);

Now let's look at the histogram of errors for the test set.

In [ ]:
errors_test = y_test - y_pred_test
plt.figure(figsize=(8,8))
plt.hist(errors_test, bins=20, color='red',  density=True)
plt.xlabel('Error in Kelvin', size=16)
plt.ylabel('Frequency', size=16)
plt.title('Linear Model Error', size=16);

## Example: K-means and Linear Regression
What if materials belong to different forms of melting? Let's separate the database into groups according to some properties. Let's take `coh_en` and `bulk_m` features (two features for visualization purposes) and use K-means algorithm to create clusters of materials. Note we will be fitting to the training set only and note that output variable is not part of unsupervised learning algorithm.

In [ ]:
n_clusters = 3 # set the number of clusters here
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X_train[:,[0,1]])

In [ ]:
train_cluster_labels = kmeans.predict(X_train[:,[0,1]])
test_cluster_labels = kmeans.predict(X_test[:,[0,1]])

In [ ]:
X_train_cluster, X_test_cluster = {}, {}
y_train_cluster, y_test_cluster = {}, {}

for i in range(n_clusters):
    idx_train = np.where(train_cluster_labels == i)[0]
    idx_test = np.where(test_cluster_labels == i)[0]
    X_train_cluster[i] = X_train[idx_train, :]
    X_test_cluster[i] = X_test[idx_test, :]
    y_train_cluster[i] = y_train[idx_train]
    y_test_cluster[i] = y_test[idx_test]

Let's visually observe if K-means did a good job.

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(n_clusters):
    plt.scatter(X_train_cluster[i][:,0], X_train_cluster[i][:,1], alpha =0.5, marker = i+4, label=f'cluster {i}')

plt.xlabel('Cohesive energy', size=16)
plt.ylabel('Bulk modulus', size=16)
plt.legend(fontsize=16);

Clustering looks good.

Now try to fit linear regression within each cluster and evaluate all error metrics as above for both training and test scores.

In [ ]:
y_pred_train_cluster, y_pred_test_cluster = {}, {}

#### <font color='RED'>YOUR SOLUTION:</font> 

Let's aid model evaluation with more visuals.

In [ ]:
plt.figure(figsize=(10, 10))

for i in range(n_clusters):
    plt.scatter(y_train_cluster[i], y_pred_train_cluster[i], color='blue', alpha =0.5, marker = i+3, label=f'train cluster {i}')
    plt.scatter(y_test_cluster[i], y_pred_test_cluster[i], color='red', alpha =0.5, marker = i+3, label=f'test cluster {i}')

plt.legend(fontsize=16);
plt.plot([-500,4500], [-500,4500], color='black')
plt.xlim([-500,4500])
plt.ylim([-500,4500])
plt.xlabel('$y_{true}$', size=16)
plt.ylabel('$y_{pred}$', size=16)

Let's look at the histogram of combined errors for the test sets of all three clusters.

In [ ]:
errors_test = []
for i in range(n_clusters):
    errors_test.extend(y_test_cluster[i] - y_pred_test_cluster[i])
    
plt.figure(figsize=(8,8))
plt.hist(errors_test, bins=20, color='red',  density=True)
plt.xlabel('Error in Kelvin', size=16)
plt.ylabel('Frequency', size=16)
plt.title('Clustering + Linear Model Error', size=16);

### Quite a few data points are off by huge margins again. How did we perform this time? Does increasing the model complexity results in a better overall model?

# Try a few other models

For ideas on what types exist, check out scikit-learn's website here:
https://scikit-learn.org/stable/user_guide.html

Searching "regression" or "clustering" in the top right corner of the page will bring up more options than initially displayed on that page. Note that most pages will have a working code example if you scroll down in case you're having difficulty getting one to work. See which types of models give better performance and consider why that might be the case. Other models can be found in different python packages if you're feeling adventurous.

# Model 1 (Regression only): _____

#### <font color='RED'>YOUR SOLUTION:</font> 

In [ ]:
# Model fitting block


In [ ]:
# Scoring block


In [ ]:
# Parity plot


# Model 2 (Clustering and Regression): _____

#### <font color='RED'>YOUR SOLUTION:</font> 

In [ ]:
# Clustering block


In [ ]:
# Model fitting block for all clusters


In [ ]:
# Scoring block for all clusters


In [ ]:
# Parity plots for all clusters


# Model 3 (Clustering and Regression vs. Regression only): _____

#### <font color='RED'>YOUR SOLUTION:</font> 

In [ ]:
# Clustering block


In [ ]:
# Model fitting block for all clusters and the whole database


In [ ]:
# Scoring block for all clusters and the whole database


In [ ]:
# Parity plots for all clusters and the whole database


In [ ]:
# Comparison of Clustering and Regression vs. Regression only (no more than 5 sentences)
